# Imitation Learning - Lab 01

O objetivo desta laboratório é experimentar a aprendizagem por imitação (imitation learning), em que o modelo (rede) aprende a imitar as ações de um especialista (humano). No lugar de experiências coletadas de um especialista humano, aqui as demonstrações serão fornecidas por meio de uma política de especialistas que treinamos para você. 

- Usaremos um tipo de aprendizado por imitação, conhecido como clonagem comportamental (behavioral cloning). Isso significa que treinaremos nossa rede de forma supervisionada.
- A saída da rede é a política de direção, representada pelo ângulo de direção desejado e/ou aceleração ou frenagem. Por exemplo, podemos ter um neurônio de saída de regressão para o ângulo de direção e um neurônio para aceleração ou frenagem (já que não podemos ter os dois ao mesmo tempo).
- A entrada da rede pode ser:
Dados brutos do sensor. Por exemplo, uma imagem da câmera. 
- Criaremos o conjunto de dados de treinamento com a ajuda do especialista. Em cada etapa da jornada, iremos registrar:
    - O estado atual do ambiente. Estes podem ser os dados brutos do sensor ou a representação da vista de cima para baixo. Usaremos o estado atual como entrada para o modelo.
    - As ações do especialista no estado atual do ambiente (ângulo de direção, freio / aceleração). Esses serão os dados de destino da rede. Durante o treinamento, vamos minimizar o erro entre as previsões da rede e as ações usando gradient descent. Desta forma, ensinaremos a rede a imitar o especialista.

<br>

A seguir está uma ilustração do cenário de Behavioral Cloning:

<br>

<img src='https://drive.google.com/uc?id=1ozI1x1hNgIa_IXNsxUm4V-YFADXlKxus' width="600" height="400">


## (Início) Configuração

Você precisará fazer uma cópia deste notebook em seu Google Drive antes de editar. Você pode fazer isso com **Arquivo → Salvar uma cópia no Drive**.

In [1]:
# !pip install google-colab > /dev/null 2>&1
# !python --version

In [2]:
import os
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Seu trabalho será armazenado em uma pasta chamada `minicurso_rl` por padrão 
# para evitar que o tempo limite da instância do Colab exclua suas edições


DRIVE_PATH = "/content/gdrive/My\ Drive/minicurso_rl"
DRIVE_PYTHON_PATH = DRIVE_PATH.replace("\\", "")
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

SYM_PATH = "/content/minicurso_rl"
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

Instalando as dependências

In [4]:
!pip install -U cloudpickle > /dev/null 2>&1 
!pip install "gym[all]" > /dev/null 2>&1 
!pip install "gym[box2d]" > /dev/null 2>&1 
!pip install "stable-baselines3[extra]" > /dev/null 2>&1 

!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

!pip install pyvirtualdisplay > /dev/null 2>&1

!pip install plotly > /dev/null 2>&1

!pip install pyarrow > /dev/null 2>&1
!pip install -U scikit-learn > /dev/null 2>&1

In [5]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/ > /dev/null 2>&1

--2021-10-16 15:29:11--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar.2’

Roms.rar.2          100%[===================>]  10.61M   636KB/s    in 18s     

2021-10-16 15:29:29 (618 KB/s) - ‘Roms.rar.2’ saved [11128004/11128004]

mkdir: cannot create directory ‘/content/ROM/’: File exists

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar


Would you like to replace the existing file /content/ROM/HC ROMS.zip
11826711 bytes, modified on 2019-12-22 11:24
with a new one
11826711 bytes, modified on 2019-12-22 11:24

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit E

No files to extract


In [6]:
!cd minicurso_rl && gdown --id 1ZV5fvCbU_gbTy1AraSR-ymNsiNQesBvt

Downloading...
From: https://drive.google.com/uc?id=1ZV5fvCbU_gbTy1AraSR-ymNsiNQesBvt
To: /content/gdrive/My Drive/minicurso_rl/EnduroNoFrameskip-v4.zip
100% 20.8M/20.8M [00:00<00:00, 97.3MB/s]


## (Sempre) Importações

In [7]:
import torch
import random
import numpy as np

# torch.multiprocessing.set_start_method('spawn')
torch.manual_seed(10)
random.seed(10)
np.random.seed(10)
eval_episodes = 10

## (Sempre) Ambiente

O ambiente utilizado será o Enduro-v0, um ambiente [OpenAI Gym](https://gym.openai.com/envs/Enduro-v0/) de corrida.

[Gym](https://gym.openai.com/docs/) é um kit de ferramentas para desenvolver e comparar algoritmos de aprendizagem por reforço. Ele não faz suposições sobre a estrutura do seu agente e é compatível com qualquer biblioteca de computação numérica.

O estado do ambiente Enduro consiste em 210x160 pixels.Uma recompensa de +1 é dada para cada carro ultrapassado e -1 para cada carro que passa pelo agente (mas a recompensa mínima é 0).

O objetivo consiste em manobrar um carro de corrida no National Enduro, uma corrida de resistência de longa distância. O objetivo da corrida é passar um certo número de carros a cada dia. Isso permitirá que o jogador continue correndo no dia seguinte. O piloto deve evitar outros pilotos e ultrapassar 200 carros no primeiro dia e 300 carros em cada dia seguinte.

Conforme o tempo passa, a visibilidade também muda. Quando é noite no jogo, o jogador só pode ver as luzes traseiras dos carros que se aproximam. Com o passar dos dias, os carros também se tornarão mais difíceis de evitar. O clima e a hora do dia são fatores importantes para jogar. Durante o dia, o jogador pode dirigir por um trecho de gelo na estrada que limitaria o controle do veículo, ou um trecho de neblina pode reduzir a visibilidade.

[Descrição da Wikipedia](https://en.wikipedia.org/wiki/Enduro_%28video_game%29)

In [8]:
# Procedimento para renderizar o ambiente no Google Colab

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()


from matplotlib import pyplot as plt, animation
%matplotlib inline
from IPython import display

def create_anim(frames, dpi, fps):
    plt.figure(figsize=(frames[0].shape[1] / dpi, frames[0].shape[0] / dpi), dpi=dpi)
    patch = plt.imshow(frames[0])
    def setup():
        plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, init_func=setup, frames=len(frames), interval=fps)
    return anim

def display_anim(frames, dpi=72, fps=60):
    anim = create_anim(frames, dpi, fps)
    return anim.to_jshtml()

def save_anim(frames, filename, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    anim.save(filename)


class trigger:
    def __init__(self):
        self._trigger = True

    def __call__(self, e):
        return self._trigger

    def set(self, t):
        self._trigger = t

In [9]:
import gym
environment_id = "EnduroNoFrameskip-v4"       # Nome do ambiente utilizado

## Visualizar

Interagimos no ambiente através da função `step`, que nos retorna quatro valores: observação, recompensa, done, info. Esta é uma implementação do clássico “loop agente-ambiente”. A cada passo de tempo, o agente escolhe uma ação e o ambiente retorna uma observação e a recompensa.
<br>

<img src='https://drive.google.com/uc?id=1TXdjYkbfm2EvtCbVIpe5BkUgXJY1d1zE' width="600" height="250">

In [10]:
env = gym.make(environment_id)                # Criando o ambiente

frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     # Retorna a observação inicial
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))     # Renderizando o ambiente
        action = env.action_space.sample()              # Seleciona uma ação aleatória
        n_obs, reward, done, info = env.step(action)    # Executa a ação selecionada
        score += reward
        obs = n_obs.copy()
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()



Episódio: 1 Pontuação: 0.0


In [ ]:
display.HTML(display_anim(frames))


## (Sempre) Carregar Modelo Especialista

O modelo especialista que estamos disponibilizando para você é um agente de aprendizado por reforço treinado com o algoritmo Proximal Policy Optmization (PPO). Para isso, foi utilizado a biblioteca [Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/), que contém uma série de implementações de algoritmos de Aprendizado por Reforço em PyTorch.

In [12]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.atari_wrappers import AtariWrapper
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack

In [13]:
expert = PPO.load("minicurso_rl/EnduroNoFrameskip-v4")

## (Sempre) Processar ambiente

Para criar o ambiente iremos aplicar alguns processamentos que irão ajudar o agente.

Os wrappers nos permitirão adicionar funcionalidade aos ambientes, como modificar observações e recompensas a serem fornecidas ao nosso agente. É comum na aprendizagem por reforço pré-processar as observações para torná-las mais fáceis de aprender. Um exemplo comum é ao usar entradas baseadas em imagem, para garantir que todos os valores estejam entre 0 e 1 ao invés de entre 0 e 255, como é mais comum com imagens RGB.

Para mais detalhes dos wrappers utilizados veja em: [Atari Wrappers](https://stable-baselines3.readthedocs.io/en/master/common/atari_wrappers.html) e [Vectorized Environments](https://stable-baselines3.readthedocs.io/en/master/guide/examples.html?highlight=make_vec_env#multiprocessing-unleashing-the-power-of-vectorized-environments).

In [14]:
env = make_vec_env(environment_id, wrapper_class=AtariWrapper)
env = VecFrameStack(env, 4)

## (Demora) Vamos ver o quão bem o especialista consegue se sair no ambiente.

In [15]:
mean_reward, std_reward = evaluate_policy(expert, env, n_eval_episodes=eval_episodes)
print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 1317.5 +/- 349.2409626604531


In [ ]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     # Retorna a observação inicial
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))       # Renderizando o ambiente
        action = expert.predict(obs, deterministic=True)  # Seleciona uma ação do agente especialista
        n_obs, reward, done, info = env.step(action)      # Executa a ação selecionada
        score += reward
        obs = n_obs.copy()
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()

display.HTML(display_anim(frames))


## Armazenar informações do especialista

Agora, deixamos nosso especialista interagir com o ambiente e armazenar as observações e ações de especialistas resultantes para construir um conjunto de dados.

In [17]:
from tqdm import tqdm

In [18]:
num_interactions = int(3e4)

In [19]:
if isinstance(env.action_space, gym.spaces.Box):
    expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
    expert_actions = np.empty((num_interactions,) + (env.action_space.shape[0],))
else:
    expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
    expert_actions = np.empty((num_interactions,) + env.action_space.shape)

# HW: Interaja  com o ambiente `env` conforme visto anteriormente. Armazene 
# as observações e as ações do especialista em `expert_observations` e 
# `expert_actions` respectivamente para construir o dataset.
obs = env.reset()     # Retorna a observação inicial
done = False
score = 0
count = 0

while not done and count < num_interactions:
    expert_observations[count] = obs.copy()
    action = expert.predict(obs, deterministic=True)  # Seleciona uma ação do agente especialista
    expert_actions[count] = action[0]
    n_obs, reward, done, info = env.step(action)      # Executa a ação selecionada
    score += reward
    obs = n_obs.copy()
    count += 1
print("\n\nPontuação: {}".format(score))
env.close()

# Salva os dados (observação, ação)
np.savez_compressed(
    "minicurso_rl/expert_data",
    expert_actions=expert_actions,
    expert_observations=expert_observations,
)



Pontuação: [1359.]


In [20]:
# Liberando memória das variáveis que não serao mais utilizadas
import gc

del expert_actions
del expert_observations
gc.collect()

2304

In [21]:
try:
    expert_observations, expert_actions
except NameError:
    pass
else:
  del expert_observations, expert_actions


# Carrega os dados salvos
data = np.load("minicurso_rl/expert_data.npz")

## (Sempre) Criando Nossos Datasets

- Para usar perfeitamente o PyTorch no processo de treinamento, criamos uma subclasse de `ExpertDataset` do `Dataset` base do Pytorch
- Observe que inicializamos o conjunto de dados com as observações e ações de especialistas geradas anteriormente.
- Implementamos ainda as [funções mágicas](https://rszalski.github.io/magicmethods/) `__getitem__` e` __len__` do Python para permitir que o manuseio do conjunto de dados do PyTorch acesse linhas arbitrárias no conjunto de dados e informá-lo sobre o comprimento do conjunto de dados.
- Para obter mais informações sobre os conjuntos de dados de PyTorch, você pode ler: https://pytorch.org/docs/stable/data.html.

In [22]:
from torch.utils.data.dataset import Dataset, random_split

In [23]:
class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions
        
    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

In [24]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

class PyArrowDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions, batch_size=128, filename="dataset.parquet"):
        # self.observations = expert_observations.copy()
        # self.actions = expert_actions.copy()
        self.batch_size = batch_size
        self.df = pd.DataFrame({
            "observations": expert_observations,
            "actions": expert_actions,
        })
        table = pa.Table.from_pandas(self.df)
        # pq.write_to_dataset(
        #     table,
        #     root_path='dataset.parquet',
        #     partition_cols=['partone', 'parttwo'],
        # )
        pq.write_table(table, filename)
        
    def __getitem__(self, index):
        _file = pq.parquet.ParquetFile(self.filename)
        batches = _file.iter_batches(batch_size) #batches will be a generator

        for batch in batches:
            process(batch)

        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

In [25]:
# Adapted from https://realpython.com/storing-images-in-python/ (Many thx!)

# !pip install h5py > /dev/null 2>&1

import h5py
from pathlib import Path

hdf5_dir = Path("data/hdf5/")
hdf5_dir.mkdir(parents=True, exist_ok=True)

def store_batch_hdf5(root, images, labels, batch_index):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        images       images array, (N, 84, 84, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    # Create a new HDF5 file
    (hdf5_dir / root).mkdir(parents=True, exist_ok=True)
    file = h5py.File(hdf5_dir / Path(root) / f"batch_{batch_index}.h5", "w")

    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(images), h5py.h5t.STD_U8BE, data=images
    )
    meta_set = file.create_dataset(
        "labels", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    file.close()

def read_batch_hdf5(root, batch_index):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        batch_index   index of batch to read

        Returns:
        ----------
        images      images array, (N, 84, 84, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / Path(root) / f"batch_{batch_index}.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/labels"]).astype("uint8")

    file.close()

    return images, labels

def generate_batch_indexes(total, batch_size, shuffle=True):
    indexes = np.arange(total)
    if True:
        rng = np.random.default_rng()
        rng.shuffle(indexes)


    num_batches = indexes.shape[0] // batch_size
    if num_batches < 1:
        raise ValueError("A quantidade de dados precisa ser maior que o tamanho do batch")
    
    batches = indexes[:batch_size * num_batches].reshape(-1, batch_size)
    for i in range(num_batches):
        yield batches[i]

In [26]:
class HDF5DataSet(Dataset):
    def __init__(self, root, batch_size=128):
        self.root = root
        self.batch_size = batch_size
        self.total_batches = 0

    def add_data(self, observations: np.ndarray, actions: np.ndarray, shuffle=True):
        assert observations.shape[0] == actions.shape[0] and observations.shape[0]

        i = 0
        for i, batch in enumerate(generate_batch_indexes(len(observations), self.batch_size, shuffle)):
            store_batch_hdf5(self.root,
                             np.take(observations, batch, axis=0),
                             np.take(actions, batch),
                             self.total_batches + i)
        self.total_batches += i+1

    def __getitem__(self, index):
        return read_batch_hdf5(self.root, index)

    def __len__(self):
        return self.total_batches

## Instanciar o Dataset Especialista

Agora instanciamos o `ExpertDataSet` e o dividimos em conjuntos de dados de treinamento e teste.

In [27]:
expert_dataset = ExpertDataSet(data["expert_observations"], data["expert_actions"])

del data

import gc
gc.collect()

train_size = int(0.8 * len(expert_dataset))     # 80% dos dados para treinamento
test_size = len(expert_dataset) - train_size    # E o restante dos dados para teste

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

In [28]:
print("# test_expert_dataset: ", len(test_expert_dataset))
print("# train_expert_dataset: ", len(train_expert_dataset))

# test_expert_dataset:  6000
# train_expert_dataset:  24000


## Treinar o agente estudante

Nossos próximos passos:

1. Extraímos a rede de políticas de nosso aluno.
2. Carregamos o conjunto de dados de especialistas (rotulados) contendo observações de especialistas como entradas e ações de especialistas como alvos.
3. Realizamos aprendizagem supervisionada, ou seja, ajustamos os parâmetros da rede de políticas de forma que, dadas as observações de especialistas como entradas para a rede, suas saídas correspondam aos alvos (ações de especialistas).


Ao treinar a rede de políticas dessa maneira, o agente aluno correspondente é ensinado a se comportar como o agente especialista que foi usado para criar o conjunto de dados especialista (Behavior Cloning).

## (Sempre) Inicializar Variáveis importantes

In [49]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.optim.lr_scheduler import StepLR

In [50]:
# Hyper Parameters
batch_size=32
epochs=20
scheduler_gamma=0.99
learning_rate=5e-3
log_interval=100
no_cuda=False    
seed=1
test_batch_size=128

In [51]:
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

## Criando a rede estudante

Agora iremos definir a rede neural que iremos utilizar para o aluno estudante. Aqui criamos um agente de aprendizado por reforço, e extraimos dele a rede da política. Alternativamente você pode construir a sua própria rede neural.

Como estamos utilizando imagens como entrada, iremos utilizar uma rede chamada de Rede Neural Convolucional (Convolutional Neural Network - CNN).

<br>

- https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-cnn-26a14c2ea29

- https://medium.com/swlh/introduction-to-cnn-image-classification-using-cnn-in-pytorch-11eefae6d83c

- https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/


In [52]:
from torchsummary import summary

student = PPO('CnnPolicy', env, verbose=1)

# Extrair politica inicial
model = student.policy.to(device)

# Mostra um sumário da rede, mostrando todas as suas camadas 
summary(model, (4, 84, 84))

Using cuda device
Wrapping the env in a VecTransposeImage.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 20, 20]           8,224
              ReLU-2           [-1, 32, 20, 20]               0
            Conv2d-3             [-1, 64, 9, 9]          32,832
              ReLU-4             [-1, 64, 9, 9]               0
            Conv2d-5             [-1, 64, 7, 7]          36,928
              ReLU-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
            Linear-8                  [-1, 512]       1,606,144
              ReLU-9                  [-1, 512]               0
        NatureCNN-10                  [-1, 512]               0
     MlpExtractor-11     [[-1, 512], [-1, 512]]               0
           Linear-12                    [-1, 1]             513
           Linear-13                    [-1,

## (Sempre) Inicializar funções de Treino

Como visto em aula, queremos minimizar a diferença entre a resposta correta e a resposta do modelo. A primeira tarefa é, portanto, definir um critério que mede o erro entre cada elemento na entrada x e no destino y.

Aqui precisamos nos atentar em alguns pontos. 

Box e Discrete são os dois tipos de espaço mais comumente usados para representar os espaços de Observação e Ação em ambientes do Gym. 

- Box: Uma caixa dimensional, onde cada coordenada fica entre um limite definido por [baixo, alto]
- Discrete: O espaço consiste em n pontos distintos, cada um mapeado para um valor inteiro no intervalo [0, n-1]


No caso do ambiente Enduro,as ações são discretas, onde será selecionado um valor entre 0 e n-1 para ser aplicado ao ambiente.

As saídas da rede é uma lista de probabilidade de selecionar cada uma dessas ações. Iremos executar a ação com a maior probabilidade dada pela rede.

<img src='https://drive.google.com/uc?id=1KEBtAKI5kOAC7PfcK3SRQIdE1sQwmAza' width="550" height="180">

Como iremos definir o erro da entrada e do destino?

O que queremos minimizar é a distância entre duas distribuições de probabilidade - prevista e real.

Considere um classificador que prediz se um dado animal é um cão, gato ou cavalo com uma probabilidade associada a cada um. 

Suponha que a imagem original seja de um cachorro e o modelo preveja 0.2, 0.7, 0.1 como probabilidade para três classes em que as probabilidades verdadeiras se parecem com 1, 0, 0. O que desejamos idealmente é que nossas probabilidades previstas sejam próximas às originais. Portanto, precisamos nos certificar de que estamos minimizando a diferença entre as duas probabilidades.

Para isso temos uma loss chamada de Cross-Entropy que nos ajuda a calcular essa diferença. Veja mais em: https://towardsdatascience.com/cross-entropy-loss-function-f38c4ec8643e

In [53]:
nb_actions = env.action_space.n
print("O número total de ações possíveis é: ", nb_actions)

O número total de ações possíveis é:  9


In [54]:
# HW: Implementar função de Loss

# from typing import List

# def get_cross_entropy_loss():
#   def cross_entropy_loss(model_out, true_out):
#     return -np.sum([true_out[i] * np.log(model_out[i]) for i in range(len(true_out))])
#   return cross_entropy_loss

# criterion = get_cross_entropy_loss()

criterion = nn.CrossEntropyLoss()

In [55]:
# HW: Implementar função de Acurácia
def acc(model_out: torch.Tensor, true_out: torch.Tensor):
    # return np.sum(model_out.detach().cpu().numpy() == true_out) / len(true_out)
    return torch.sum(torch.argmax(model_out, dim=1).eq(true_out))

In [56]:
# HW: Implementar função de Treino
# ela deve retornar informações de loss e acurácia

def train():
    _loss = 0.0
    _acc = 0.0

    model.train()
    for data, target in train_loader:
        data = data.permute(0, 3, 1, 2)
        data, target = data.to(device), target.to(device)

        if isinstance(env.action_space, gym.spaces.Box):
            # A2C/PPO policy outputs actions, values, log_prob
            action, _, _ = model(data)
            action_prediction = action.double()
        else:
            # Retrieve the logits for A2C/PPO when using discrete actions
            latent_pi, _, _ = model._get_latent(data)
            logits = model.action_net(latent_pi)
            action_prediction = logits
            target = target.long()

        train_loss = criterion(action_prediction, target)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        _loss += train_loss.data.cpu().numpy()
        _acc += acc(action_prediction, target).data.cpu().numpy()
    _loss /= float(len(train_loader.dataset))
    _acc /= float(len(train_loader.dataset))

    print(f"Conjunto de Treino: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")
    return _loss, _acc

In [57]:
def test():
    _loss = 0.0
    _acc = 0.0

    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data = data.permute(0, 3, 1, 2)
            data, target = data.to(device), target.to(device)

            if isinstance(env.action_space, gym.spaces.Box):
                # A2C/PPO policy outputs actions, values, log_prob
                action, _, _ = model(data)
                action_prediction = action.double()
            else:
                # Retrieve the logits for A2C/PPO when using discrete actions
                latent_pi, _, _ = model._get_latent(data)
                logits = model.action_net(latent_pi)
                action_prediction = logits
                target = target.long()
            
            test_loss = criterion(action_prediction, target)

            _loss += test_loss.data.cpu().numpy()
            _acc += acc(action_prediction, target).data.cpu().numpy()

    _loss /= float(len(test_loader.dataset))
    _acc /= float(len(test_loader.dataset))
    print(f"Conjunto de Teste: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")
    return _loss, _acc

## (Demora) Avaliar agente antes do Treino

Avalie o agente antes do treinamento (seu comportamento deve ser aleatório)

In [38]:
mean_reward, std_reward = evaluate_policy(student, env, n_eval_episodes=eval_episodes)
print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 0.0 +/- 0.0


## Treinar agente

In [58]:
# Aqui, usamos PyTorch `DataLoader` para carregar o` ExpertDataset` criado anteriormente para treinamento e teste
train_loader = torch.utils.data.DataLoader(
    dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
)

# Defina um Otimizador e uma programação de taxa de aprendizagem (learning rate).
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

Tendo definido o procedimento de treinamento, podemos agora executar o treinamento!

In [59]:
# Agora estamos finalmente prontos para treinar o modelo de política.
train_loss, train_acc = [], []
test_loss, test_acc = [], []
_learning_rate = []

for epoch in range(1, epochs + 1):
    _learning_rate.append(scheduler.get_last_lr()[0])
    print("learning rate: ", scheduler.get_last_lr()[0])

    _train_loss, _train_acc = train()
    _test_loss, _test_acc = test()

    train_loss.append(_train_loss)
    train_acc.append(_train_acc)
    test_loss.append(_test_loss)
    test_acc.append(_test_acc)
    
    scheduler.step()

learning rate:  0.005
Conjunto de Treino: Loss 0.0217 	Accuracy 74.08 %
Conjunto de Teste: Loss 0.0045 	Accuracy 76.83 %
learning rate:  0.00495
Conjunto de Treino: Loss 0.0182 	Accuracy 77.52 %
Conjunto de Teste: Loss 0.0044 	Accuracy 78.13 %
learning rate:  0.0049005
Conjunto de Treino: Loss 0.0163 	Accuracy 79.96 %
Conjunto de Teste: Loss 0.0043 	Accuracy 78.95 %
learning rate:  0.004851495
Conjunto de Treino: Loss 0.0149 	Accuracy 81.56 %
Conjunto de Teste: Loss 0.0041 	Accuracy 81.13 %
learning rate:  0.00480298005
Conjunto de Treino: Loss 0.0138 	Accuracy 83.12 %
Conjunto de Teste: Loss 0.0039 	Accuracy 81.35 %
learning rate:  0.0047549502495
Conjunto de Treino: Loss 0.0129 	Accuracy 83.71 %
Conjunto de Teste: Loss 0.0039 	Accuracy 81.70 %
learning rate:  0.004707400747005
Conjunto de Treino: Loss 0.0122 	Accuracy 85.07 %
Conjunto de Teste: Loss 0.0041 	Accuracy 82.43 %
learning rate:  0.00466032673953495
Conjunto de Treino: Loss 0.0115 	Accuracy 85.82 %
Conjunto de Teste: Loss 0

## Visualizar Gráficos

In [60]:
import plotly.graph_objs as go

In [61]:
fig = go.Figure([
    go.Scatter(
        y=train_acc,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Acurácia de Treino"
    ),
    go.Scatter(
        y=test_acc,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Acurácia de Teste"
    ),
])
fig.update_layout(
    title="Acurácia",
    yaxis = dict(
        tickformat = "%",
    ),
    xaxis = dict(
        title = "Época",
    )
)
fig.show()

In [62]:
fig = go.Figure([
    go.Scatter(
        y=train_loss,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Loss de Treinamento"
    ),
    go.Scatter(
        y=test_loss,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Loss de Teste"
    ),
])
fig.update_layout(
    title="Loss",
    xaxis = dict(title="Época")
)
fig.show()

In [63]:
fig = go.Figure([
    go.Scatter(
        y=_learning_rate,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Learning Rate"
    ),
])
fig.update_layout(
    title="Learning Rate",
    xaxis = dict(title="Época")
)
fig.show()

Finalmente, vamos testar o quão bem nosso aluno aprendeu a imitar o comportamento do especialista

In [64]:
# Inserir a rede treinada de volta no agente estudante
student.policy = model

In [65]:
mean_reward, std_reward = evaluate_policy(student, env, n_eval_episodes=eval_episodes)

print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 10.8 +/- 14.600000000000001


In [66]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()     
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))           
        action = student.predict(obs, deterministic=True)
        n_obs, reward, done, info = env.step(action)       
        score += reward
        obs = n_obs.copy()
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()



Episódio: 1 Pontuação: [0.]


In [ ]:
display.HTML(display_anim(frames))

# Bônus

O algoritimo Dagger é um algoritimo interativo que aproxima as distribuições de trajetórias de alunos e especialistas ao rotular pontos de dados adicionais resultantes da aplicação da política atual.

Em sua forma mais simples, o algoritmo procede da seguinte maneira. Na primeira iteração, ele usa a política do especialista para reunir um conjunto de dados de trajetórias $D$ e treinar uma política $\pi_{2}$ que melhor imita o especialista nessas trajetórias. Então, na iteração $n$, ele usa $\pi_{n}$ para coletar mais trajetórias e adiciona essas trajetórias ao conjunto de dados $D$. A próxima política $\pi_{n+1}$ é a que melhor imita o especialista em todo o conjunto de dados $D$.

É como se a cada passo, perguntássemos ao especialista sua opinião sobre nossa trajetória atual. Em seguida, reunindo esta opinião (sua resposta aos estados que encontramos) e os conjuntos de dados anteriores de trajetórias,
podemos treinar uma nova política mais precisa porque estamos levando em consideração a opinião de mais especialistas.

<br>

Algoritmo [Dagger](http://proceedings.mlr.press/v15/ross11a/ross11a.pdf):

<img src='https://drive.google.com/uc?id=1rMERL80AGmDRR0fKVfq0KjhJjGPt4YKt' width="450" height="250">


A tarefa bônus consistirá em implementar o algoritmo Dagger.


## (Sempre e antes de reiniciar treinos) Defining basic functions and reinitializing (Hyper)parameters and variables

In [164]:
episodes_per_train = 2
batch_size = 128
batch_size_test = 128
num_interactions = int(2.5e3)

In [165]:
dagger_student = PPO('CnnPolicy', env, verbose=1)
dagger_model = dagger_student.policy.to(device)
expert_model = expert.policy.to(device)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [166]:
def prepare_obs(obs):
  return torch.from_numpy(obs).permute(0, 3, 1, 2).to(device)

In [167]:
def get_merged_action(expert_model, dagger_model, beta, obs):
    if np.random.random_sample() < beta:
        return expert_model.predict(obs, deterministic=True)
    return dagger_model.predict(obs, deterministic=True)
    #     return expert_model(prepare_obs(obs), deterministic=True)
    # return dagger_model(prepare_obs(obs), deterministic=True)

In [168]:
# This is equivalent to run the game one time
def get_merged_trajectory(expert_model, dagger_model, size, env, beta):
    trajectory = np.empty((num_interactions, ) + env.observation_space.shape)
    obs = env.reset()
    done = False
    count = 0
    while not done and count < size:
        trajectory[count] = obs.copy()
        action, _ = get_merged_action(expert_model, dagger_model, beta, obs)
        next_obs, reward, done, info = env.step(action)
        obs = next_obs.copy()
        count += 1
    return trajectory

def generate_merged_trajectory(expert_model, dagger_model, size, env, beta, batch_size):
    trajectory = np.empty((batch_size, ) + env.observation_space.shape)
    obs = env.reset()
    done = False
    count = 0
    while not done and count < size:
        trajectory[count] = obs.copy()
        action, _ = get_merged_action(expert_model, dagger_model, beta, obs)
        next_obs, reward, done, info = env.step(action)
        obs = next_obs.copy()
        count += 1
        count %= batch_size
        if count == batch_size-1:
            yield trajectory.copy()

In [182]:
def get_random_trajectory(model, env, size=num_interactions, sample=0.2):
    rng = np.random.default_rng()
    trajectory = np.empty((size,) + env.observation_space.shape)
    obs = env.reset()
    done = False
    count = 0
    while not done and count < size:
        trajectory[count] = obs.copy()
        action, _ = model.predict(obs, deterministic = True)
        next_obs, reward, done, info = env.step(action)
        obs = next_obs.copy()
        count += 1
    return rng.choice(trajectory, size=int(size*sample), axis=0).copy()

def generate_random_trajectory(model, env, size=num_interactions, batch_size=128):
    rng = np.random.default_rng()
    trajectory = np.empty((batch_size,) + env.observation_space.shape)
    obs = env.reset()
    done = False
    count = 0
    while not done and count < size:
        trajectory[count] = obs.copy()
        action, _ = model.predict(obs, deterministic = True)
        next_obs, reward, done, info = env.step(action)
        obs = next_obs.copy()
        count += 1
        count %= batch_size
        if count == batch_size-1:
            yield rng.choice(trajectory, size=batch_size, axis=0)

In [188]:
# def get_actions(model, trajectories):
#   actions = []
#   for t in trajectories:
#     a, _, _ = model(prepare_obs(t), deterministic=True)
#     actions.append(a)
#   return actions

def get_actions(model, trajectory):
    a, _ = model.predict(trajectory, deterministic=True)
    return a

In [171]:
class SchedulerBeta:
  def __init__(self, initial_beta = 1.0, decay_rate = 0.9):
      self.initial_beta = initial_beta
      self.decay_rate = decay_rate
      self.reset()

  def get_beta(self):
    return self.beta

  def reset(self):
      self.beta = self.initial_beta
      self.i = 0
      return self.beta

  def step(self):
      self.beta *= self.decay_rate
      self.i += 1
      return self.beta

In [172]:
train_loss, train_acc = [], []
test_loss, test_acc = [], []
_learning_rate = []

optimizer = optim.Adam(dagger_model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

In [173]:
import os
min_val_loss = np.inf
MODEL_PATH = "model/"
if not os.path.exists(MODEL_PATH):
  %mkdir $MODEL_PATH
DAGGER_MODEL_NAME = "dagger.model"

def check_save(model, val_loss, name=DAGGER_MODEL_NAME):
    global min_val_loss
    if val_loss < min_val_loss:
        min_val_loss = val_loss.copy()
        path = MODEL_PATH + name
        model.save(path)
        print(f"Salvando melhor modelo: {name}\tValidation loss: {val_loss}")

In [174]:
def run_episode(pi, loader, train=True):
    _loss = 0.0
    _acc = 0.0
    name = 'treino' if train else 'teste'

    for data, target in loader:
        # Adaptation needed to run model with HDF5 Loadings
        # To reajust shapes to correct values
        # data = data.reshape(-1, 84, 84, 4)
        data = data.reshape((-1,) + env.observation_space.shape)
        target = target.flatten()

        data = data.permute(0, 3, 1, 2)
        data, target = data.to(device), target.to(device)

        if isinstance(env.action_space, gym.spaces.Box):
            # A2C/PPO policy outputs actions, values, log_prob
            action, _, _ = pi(data)
            action_prediction = action.double()
        else:
            # Retrieve the logits for A2C/PPO when using discrete actions
            latent_pi, _, _ = pi._get_latent(data)
            logits = pi.action_net(latent_pi)
            action_prediction = logits
            target = target.long()

        loss = criterion(action_prediction, target)
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        _loss += loss.data.cpu().numpy()
        _acc += acc(action_prediction, target).data.cpu().numpy()
    _loss /= float(len(loader.dataset) * batch_size)
    _acc /= float(len(loader.dataset) * batch_size)

    print(f"Resultado do {name}: Loss {_loss:.4f} \tAccuracy {_acc*100:.2f} %")

    if not train:
        check_save(pi, _loss)
        
    return _loss, _acc

In [175]:
def train_dagger(dagger_model, train_loader, test_loader):
    for episode in range(1, episodes_per_train + 1):
        _learning_rate.append(scheduler.get_last_lr()[0])
        print("learning rate: ", scheduler.get_last_lr()[0])

        _train_loss, _train_acc = run_episode(dagger_model, train_loader, train=True)
        with torch.no_grad():
          _test_loss, _test_acc = run_episode(dagger_model, test_loader, train=False)

        train_loss.append(_train_loss)
        train_acc.append(_train_acc)
        test_loss.append(_test_loss)
        test_acc.append(_test_acc)
        
        scheduler.step()

## Basic Dagger implementation

In [160]:
N = 8
num_interactions = int(2.5e3)

beta_scheduler = SchedulerBeta()

trajectory, actions = None, None

test_trajectory = get_random_trajectory(expert_model, env, size=num_interactions, sample=0.2)
test_actions = get_actions(expert_model, test_trajectory)
test_dataset = ExpertDataSet(test_trajectory, test_actions)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size_test, shuffle=True
)
del test_trajectory, test_actions, test_dataset

# Running the algorithm
D_obs = np.array([]).reshape((0, ) + env.observation_space.shape)
D_actions = np.array([])
for i in range(1, N+1):
    print(f"Episode: {i}")
    print(f"beta: {beta_scheduler.get_beta()}")
    trajectory = get_merged_trajectory(expert_model,
                                       dagger_model,
                                       num_interactions,
                                       env,
                                       beta_scheduler.get_beta())
    actions = get_actions(expert_model, trajectory)
    D_obs = np.vstack([D_obs, trajectory])
    D_actions = np.append(D_actions, actions)

    train_dataset = ExpertDataSet(D_obs, D_actions)
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=batch_size, shuffle=True, **kwargs
    )
    del train_dataset, actions

    train_dagger(dagger_model, train_loader, test_loader)
    beta_scheduler.step()
    print("\n")
del D_obs, D_actions
env.close()

Episode: 1
beta: 1.0
learning rate:  0.0042572888554743785
Resultado do treino: Loss 0.0000 	Accuracy 0.69 %
Resultado do teste: Loss 0.0002 	Accuracy 0.31 %
learning rate:  0.004214715966919635
Resultado do treino: Loss 0.0000 	Accuracy 0.69 %
Resultado do teste: Loss 0.0002 	Accuracy 0.29 %


Episode: 2
beta: 0.9
learning rate:  0.004172568807250438
Resultado do treino: Loss 0.0000 	Accuracy 0.63 %
Resultado do teste: Loss 0.0002 	Accuracy 0.34 %
learning rate:  0.0041308431191779335
Resultado do treino: Loss 0.0000 	Accuracy 0.65 %
Resultado do teste: Loss 0.0002 	Accuracy 0.36 %


Episode: 3
beta: 0.81
learning rate:  0.004089534687986154
Resultado do treino: Loss 0.0000 	Accuracy 0.59 %
Resultado do teste: Loss 0.0002 	Accuracy 0.30 %
learning rate:  0.004048639341106292
Resultado do treino: Loss 0.0000 	Accuracy 0.63 %
Resultado do teste: Loss 0.0002 	Accuracy 0.32 %


Episode: 4
beta: 0.7290000000000001
learning rate:  0.00400815294769523
Resultado do treino: Loss 0.0000 	Accura

## Dagger with K-fold Cross Validation (not implemented yet)

In [ ]:
# from sklearn.model_selection import KFold

# N = 8
# num_interactions = int(2.5e3)
# beta_scheduler = SchedulerBeta()

# trajectory, actions = None, None

# # test_trajectory = get_random_trajectory(expert_model, env, size=num_interactions, sample=0.2)
# # test_actions = get_actions(expert_model, test_trajectory)
# # test_dataset = ExpertDataSet(test_trajectory, test_actions)
# # test_loader = torch.utils.data.DataLoader(
# #     dataset=test_dataset, batch_size=batch_size_test, shuffle=True
# # )
# # del test_trajectory, test_actions, test_dataset

# # Running the algorithm
# D_obs = np.array([]).reshape((0, ) + env.observation_space.shape)
# D_actions = np.array([])
# for i in range(1, N+1):
#     print(f"Episode: {i}")
#     print(f"beta: {beta_scheduler.get_beta()}")
#     trajectory = get_merged_trajectory(expert_model,
#                                        dagger_model,
#                                        num_interactions,
#                                        env,
#                                        beta_scheduler.get_beta())
#     actions = get_actions(expert_model, trajectory)
#     D_obs = np.vstack([D_obs, trajectory])
#     D_actions = np.append(D_actions, actions.cpu().numpy())

#     train_dataset = ExpertDataSet(D_obs, D_actions)
#     train_loader = torch.utils.data.DataLoader(
#         dataset=train_dataset, batch_size=batch_size, shuffle=True, **kwargs
#     )
#     del train_dataset, actions

#     train_dagger(dagger_model, train_loader, test_loader)
#     beta_scheduler.step()
#     print("\n")
# del D_obs, D_actions
# env.close()


## Dagger with large Dataset

In [189]:
N = 8
num_interactions = int(1e6)

episodes_per_train = 2

beta_scheduler = SchedulerBeta()

train_dataset = HDF5DataSet('train', batch_size)
test_dataset = HDF5DataSet('test', batch_size_test)

for test_trajectory in generate_random_trajectory(expert_model,
                                                  env,
                                                  size=num_interactions,
                                                  batch_size=batch_size_test):
    test_actions = get_actions(expert_model, test_trajectory)
    test_dataset.add_data(test_trajectory, test_actions, shuffle=False)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=1, shuffle=False
)

# Running the algorithm
D_obs = np.array([]).reshape((0, ) + env.observation_space.shape)
D_actions = np.array([])
for i in range(1, N+1):
    print(f"Episode: {i}")
    print(f"beta: {beta_scheduler.get_beta()}")

    for trajectory in generate_merged_trajectory(expert_model,
                                                 dagger_model,
                                                 num_interactions,
                                                 env,
                                                 beta_scheduler.get_beta(),
                                                 batch_size):
        actions = get_actions(expert_model, trajectory)
        train_dataset.add_data(trajectory, actions, shuffle=True)
    
    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=1, shuffle=True, **kwargs
    )

    train_dagger(dagger_model, train_loader, test_loader)
    beta_scheduler.step()
    print("\n")
del D_obs, D_actions
env.close()

Episode: 1
beta: 1.0
learning rate:  0.005
Resultado do treino: Loss 0.0191 	Accuracy 36.98 %
Resultado do teste: Loss 0.0127 	Accuracy 32.03 %
Salvando melhor modelo: dagger.model	Validation loss: 0.012671315044696842
learning rate:  0.00495
Resultado do treino: Loss 0.0125 	Accuracy 44.05 %
Resultado do teste: Loss 0.0133 	Accuracy 32.65 %


Episode: 2
beta: 0.9


KeyboardInterrupt: ignored

## Loading Model

In [114]:
dagger_model.load(MODEL_PATH + DAGGER_MODEL_NAME)

ActorCriticCnnPolicy(
  (features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=3136, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential()
    (value_net): Sequential()
  )
  (action_net): Linear(in_features=512, out_features=9, bias=True)
  (value_net): Linear(in_features=512, out_features=1, bias=True)
)

## Gerar Gráficos

In [115]:
import plotly.graph_objs as go

In [116]:
fig = go.Figure([
    go.Scatter(
        y=train_acc,
        x=[ep for ep in range(1, len(train_acc) + 1)],
        mode='lines',
        name="Acurácia de Treino"
    ),
    go.Scatter(
        y=test_acc,
        x=[ep for ep in range(1, len(test_acc) + 1)],
        mode='lines',
        name="Acurácia de Teste"
    ),
])
fig.update_layout(
    title="Acurácia",
    yaxis = dict(
        tickformat = "%",
    ),
    xaxis = dict(
        title = "Época",
    )
)
fig.show()

In [117]:
fig = go.Figure([
    go.Scatter(
        y=train_loss,
        x=[ep for ep in range(1, len(train_loss) + 1)],
        mode='lines',
        name="Loss de Treinamento"
    ),
    go.Scatter(
        y=test_loss,
        x=[ep for ep in range(1, len(test_loss) + 1)],
        mode='lines',
        name="Loss de Teste"
    ),
])
fig.update_layout(
    title="Loss",
    xaxis = dict(title="Época")
)
fig.show()

In [118]:
fig = go.Figure([
    go.Scatter(
        y=_learning_rate,
        x=[ep for ep in range(1, epochs + 1)],
        mode='lines',
        name="Learning Rate"
    ),
])
fig.update_layout(
    title="Learning Rate",
    xaxis = dict(title="Época")
)
fig.show()

## (Demora) Testar agente

In [119]:
# Evaluate Dagger
mean_reward, std_reward = evaluate_policy(dagger_student, env, n_eval_episodes=eval_episodes)

print(f"Recompensa média = {mean_reward} +/- {std_reward}")

Recompensa média = 100.0 +/- 100.0


## Visualizar Frames

In [120]:
frames = []
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        frames.append(env.render(mode='rgb_array'))
        action = dagger_student.predict(obs, deterministic=True)
        n_obs, reward, done, info = env.step(action)
        score += reward
        obs = n_obs.copy()
    print("\n\nEpisódio: {} Pontuação: {}".format(episode,score))
env.close()



Episódio: 1 Pontuação: [0.]


In [ ]:
display.HTML(display_anim(frames))